In [1]:
! pip install opencv-python albumentations

In [2]:
import cv2
import albumentations as A
import os

dataset_path = 'C:\\Users\\KHU\\workspace_mj\\X_ray\\data_ver2\\train_aug\\'

target_class_ids = (1, 2, 3, 4, 5, 6, 7, 8, 12, 15, 17, 18, 19, 21, 22, 24, 25, 26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64,65)

transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=90, p=0.5),
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

for filename in os.listdir(dataset_path):
    if filename.endswith('.png'):
        image_path = os.path.join(dataset_path, filename)
        txt_path = os.path.splitext(image_path)[0] + '.txt'

        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        bboxes = []
        class_labels = []
        with open(txt_path, 'r') as file:
            for line in file:
                class_id, x_center, y_center, width, height = map(float, line.split())

                if class_id in target_class_ids:
                    bboxes.append([x_center, y_center, width, height])
                    class_labels.append(class_id)

        if bboxes:
            transformed = transform(image=image, bboxes=bboxes, class_labels=class_labels)
            transformed_image = transformed['image']
            transformed_bboxes = transformed['bboxes']

            transformed_image_path = os.path.splitext(image_path)[0] + '_augmented.jpg'
            transformed_image = cv2.cvtColor(transformed_image, cv2.COLOR_RGB2BGR)
            cv2.imwrite(transformed_image_path, transformed_image)

            transformed_txt_path = os.path.splitext(txt_path)[0] + '_augmented.txt'
            with open(transformed_txt_path, 'w') as file:
                for bbox, class_id in zip(transformed_bboxes, class_labels):
                    file.write(f"{class_id} {' '.join(map(str, bbox))}\n")